In [1]:
import numpy as np
import pandas as pd
from itertools import combinations
from scipy.special import comb

### Explanation of the technique being used, and checking that it works.

In [2]:
# https://docs.python.org/3/library/itertools.html#itertools.combinations
# The combination tuples are emitted in lexicographic ordering according to the order of the input iterable. 
# So, if the input iterable is sorted, the combination tuples will be produced in sorted order.

# What this means for our purposes is that say we need a maximum of 4 IDs sampled in each iteration.
# The IDs for one particular iteration might be example_ids=[9, 1, 4, 5].
# This list is now considered sorted as being in the order that it was randomly drawn, and we keep it that way.
# Given the above documentation, the order of pairs produced by combinations(example_ids, 2) is guaranteed.
# [(9, 1), (9, 4), (9, 5), (1, 4), (1, 5), (4, 5)]

# The pairs (tuples) are in lexicographic ordering as described, and the combinations are ordered based on the input.
# This means that we can take the last 'n choose k' combinations as all possible combinations that can be formed
# for only the last n IDs selected. So if we only want to look up random pairs for n=3, we can say that the randomly
# sampled IDs in this iteration were (1,4,5), and n (3) choose k (always 2) gives us 3, so we take the final three 
# tuples in the produced list, which are the pairs (1,4) (1,5), and (4,5). That works for any value of n.

In [3]:
# Demonstration of the above explanation.
ids = [9, 1, 4, 5]
id_pairs = list(combinations(ids, 2))
print("This is all the IDs randomly sampled in this iteration, supporting a value of n up to {}.".format(len(ids)))
print(ids)
print(id_pairs)
print()

n=3
n_choose_k = comb(n,2,exact=True)
n_random_ids = ids[len(ids)-n:len(ids)]
all_pairs_for_n_random_ids = id_pairs[len(id_pairs)-n_choose_k:len(id_pairs)]
print("If we need to check a random sample of fewer than {} IDs, we can take the last n values of the list.".format(len(ids)))
print(n_random_ids)
print(all_pairs_for_n_random_ids)

This is all the IDs randomly sampled in this iteration, supporting a value of n up to 4.
[9, 1, 4, 5]
[(9, 1), (9, 4), (9, 5), (1, 4), (1, 5), (4, 5)]

If we need to check a random sample of fewer than 4 IDs, we can take the last n values of the list.
[1, 4, 5]
[(1, 4), (1, 5), (4, 5)]


### Now we want to apply that process to a large amount of random iterations using numpy array.

In [4]:
# This is the number of IDs, and the dimensions of a 2D array that contains pairwise distances as floats.
dim = 1000 
distances = np.random.random((dim,dim))
distances.shape

(1000, 1000)

In [5]:
# These are the actual IDs, just making them the same as their indices in the array for simplicity.
ids = list(range(dim))

In [6]:
# what's the maximum number of genes that we might want to randomly select?
# For our purposes this value is around 50, this approach will scale very well for very large values of n.
n_max = 50

In [7]:
# How many random samplings do we want to do for calculating p-values?
num_iter = 100

In [8]:
# Create an array that contains an array of randomly sampled IDs for each iteration.
sampled_ids = np.array([np.random.choice(a=ids, size=n_max, replace=False) for i in range(num_iter)])
sampled_ids.shape

(100, 50)

In [9]:
# Create an array that allows us to look up pairs of IDs really fast.
sampled_id_pairs = np.array([list(combinations(id_list,2)) for id_list in sampled_ids])
sampled_id_pairs.shape

(100, 1225, 2)

In [10]:
# The first dimension is the iteration in the sampling we're doing.
# The second dimension are the pairs among all the possible pairs you can make.
# The third is of length two and is the actual identify of the two nodes in terms of their ID.
# So,
#sampled_id_pairs[50,235] # gives you the IDs in the 235th pair sampled during the 50th iteration.

In [11]:
# Using the technique described at the beginning, we should be able to use n choose k to get the list of edges for
# some other value of n (less than or equal to n_max) using just a single slice of that array.
length = sampled_id_pairs.shape[1]
n = 30
num_pairs_to_take = comb(n, k=2, exact=True)
sampled_id_pairs_for_other_n = sampled_id_pairs[:, length-num_pairs_to_take:length]
sampled_id_pairs_for_other_n.shape

(100, 435, 2)

In [12]:
# Now create a corresponding array, but this time use the indices to look up distances so it's just a 2D array.
sampled_values = []
for pairs_of_ids in sampled_id_pairs:
    sampled_values.append([distances[i,j] for (i,j) in pairs_of_ids])
sampled_values = np.array(sampled_values)
sampled_values.shape

(100, 1225)

In [13]:
# Again, we can use the  technique described at the beginning, we should be able to use n choose k to get the list 
# of values for some other value of n (less than or equal to n_max) using just a single slice of that array.
length = sampled_values.shape[1]
n = 30
num_values_to_take = comb(n, k=2, exact=True)
sampled_values_for_other_n = sampled_values[:, length-num_values_to_take:length]
sampled_values_for_other_n.shape

(100, 435)

In [14]:
# However, the ultimate goal is to just provide a value of n and get back the mean of all the edges,
# So we can reduce this array further to only contain the values that we'd ever want to use.

# First, create a list of all the n choose k outputs given out n max. This is equivalent to the slice sizes we need.
n_to_n_choose_two = {}
n = 2
while n <= n_max:
    n_to_n_choose_two[n] = comb(n, k=2, exact=True)
    n = n+1
    
    

n_to_means = {}
length = sampled_values.shape[1]
for n,n_choose_two in n_to_n_choose_two.items():
    num_values_to_take = n_choose_two
    means = np.mean(sampled_values[:, length-num_values_to_take:length], axis=1)
    n_to_means[n] = means
    
    
n_to_means.keys()
n_to_means.values()

dict_values([array([0.67554164, 0.37840595, 0.27945468, 0.10034196, 0.85779332,
       0.47112885, 0.04918113, 0.08656534, 0.81739856, 0.00612362,
       0.18635859, 0.12644729, 0.65049656, 0.703592  , 0.16716504,
       0.84567249, 0.70699484, 0.34660235, 0.72118253, 0.54436352,
       0.88033804, 0.20235436, 0.75913234, 0.82555478, 0.10698472,
       0.13844198, 0.17230412, 0.18747169, 0.35954871, 0.57842022,
       0.10260905, 0.17584628, 0.10828836, 0.11470966, 0.04153017,
       0.20216547, 0.79776825, 0.23500888, 0.49112591, 0.46847314,
       0.13382982, 0.04112145, 0.84739017, 0.2304194 , 0.29691231,
       0.90891889, 0.43298821, 0.47804545, 0.07801502, 0.61672944,
       0.72830274, 0.78635185, 0.44247657, 0.32366561, 0.23856914,
       0.16024189, 0.09836174, 0.59207272, 0.33870639, 0.42904606,
       0.55406325, 0.66377492, 0.33148066, 0.97801067, 0.30026414,
       0.02648276, 0.64455232, 0.08393891, 0.40168822, 0.8424406 ,
       0.7762032 , 0.30248476, 0.68561932, 0.6789